In [44]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import matplotlib.pyplot as plt
from cassandra.cluster import Cluster
import requests

In [45]:
pip install cassandra-driver


Note: you may need to restart the kernel to use updated packages.


Connect to Cassandra

In [142]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

Create keyspace

In [47]:
keyspace_name = "data_quran"  


session.execute("CREATE KEYSPACE IF NOT EXISTS data_quran WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}")

session.set_keyspace('data_quran')

use keyspace

In [48]:
session.set_keyspace('data_quran')

create table

In [134]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS arabic (
        verse_number int PRIMARY KEY,
        surah int,
        ayah_id int,
        text varchar
    );
"""

try:
    session.execute(create_table_query)
    print("Table 'arabic' created successfully!")
except Exception as e:
    print("Error:", e)

Table 'arabic' created successfully!


In [138]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS Tafseer (
        Name varchar,
        Surah int,
        Ayat int,
        Verse text,
        Tafseer text,
        PRIMARY KEY (Name, Surah, Ayat)
    );
"""

try:
    session.execute(create_table_query)
    print("Table 'Tafseer' created successfully!")
except Exception as e:
    print("Error: Issue creating table")
    print(e)

Table 'Tafseer' created successfully!


Extract data from csv

In [135]:
# Path to your CSV file
csv_file = r"C:\Users\ASUS\Desktop\big-data-engineering-with-python-2023\final_project\data\arabic_ayah.csv"  # Replace with your CSV file path

# Read data from CSV and insert into Cassandra
try:
    # Read CSV file into DataFrame
    df = pd.read_csv(csv_file)
    
    # Iterate over DataFrame rows and insert into Cassandra
    for row in df.itertuples(index=False):
        # Prepare and execute INSERT query
        insert_query = session.prepare("INSERT INTO arabic (verse_number, surah, ayah_id, text) VALUES (?, ?, ?, ?)")
        session.execute(insert_query, (row.verse_number, row.surah, row.ayah_id, row.text))
        
    print("Data inserted successfully!")
except Exception as e:
    print("Error: Inserting Rows")
    print(e)


Data inserted successfully!


In [140]:



file_path = r"C:\Users\ASUS\Desktop\big-data-engineering-with-python-2023\final_project\data\Quran_English_with_Tafseer.csv"

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        next(file) 
        for line in file:
            parts = line.strip().split(',')  # Assuming data is CSV
            name = parts[0]
            surah = int(parts[1])
            ayat = int(parts[2])
            verse = parts[3]
            tafseer = parts[4]
            # Prepare and execute INSERT query
            insert_query = session.prepare("INSERT INTO Tafseer (Name, Surah, Ayat, Verse, Tafseer) VALUES (?, ?, ?, ?, ?)")
            session.execute(insert_query, (name, surah, ayat, verse, tafseer))
    print("Data inserted successfully!")
except Exception as e:
    print("Error: Inserting Data")
    print(e)

# Close connection
cluster.shutdown()


Data inserted successfully!


Show Al-quran data 

In [136]:
keyspace_name = "data_quran"
session.set_keyspace(keyspace_name)


table_name = "arabic"


rows = session.execute(f"SELECT * FROM {table_name} LIMIT 1")


for row in rows:
    print(row)  # 

Row(verse_number=4317, ayah_id=45, surah=42, text='وَتَرَىٰهُمۡ يُعۡرَضُونَ عَلَيۡهَا خَٰشِعِينَ مِنَ ٱلذُّلِّ يَنظُرُونَ مِن طَرۡفٍ خَفِيّٖۗ وَقَالَ ٱلَّذِينَ ءَامَنُوٓاْ إِنَّ ٱلۡخَٰسِرِينَ ٱلَّذِينَ خَسِرُوٓاْ أَنفُسَهُمۡ وَأَهۡلِيهِمۡ يَوۡمَ ٱلۡقِيَٰمَةِۗ أَلَآ إِنَّ ٱلظَّـٰلِمِينَ فِي عَذَابٖ مُّقِيمٖ')


The Question

In [144]:
keyspace_name = "data_quran"
session.set_keyspace(keyspace_name)


table_name = "Tafseer"


rows = session.execute(f"SELECT * FROM {table_name} LIMIT 1")


for row in rows:
    print(row)  # 

Row(name='The Reality', surah=69, ayat=1, tafseer='The Reality the Resurrection in which is realised the truth of all that was rejected in the way of the raising from the graves the reckoning and the requital or it means the Resurrection which will manifest all of that.', verse='The Reality!')
